# Classifier Performance

In [ ]:
from pathlib import Path

import pandas as pd
from sklearn.metrics import f1_score, roc_curve, precision_recall_curve, auc

In [ ]:
exp_dir = Path("/opt/gpudata/labrag")

dataset_models = [("mimic", "biovilt"), ("chexpertplus", "gloria")]
sections = ["findings", "impression"]
labelers = ["chexbert", "chexpert"]

labels = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Enlarged Cardiomediastinum",
    "Fracture",
    "Lung Lesion",
    "Lung Opacity",
    "No Finding",
    "Pleural Effusion",
    "Pleural Other",
    "Pneumonia",
    "Pneumothorax",
    "Support Devices",
]

results = []
for dataset, base_model in dataset_models:
    for section in sections:
        for model in [base_model, "resnet50"]:
            for labeler in labelers:
                cls_dir = exp_dir / f"{dataset}-{section}-{model}-classifiers-{labeler}"
                df_true = pd.read_csv(cls_dir / "test_true.csv")
                df_prob = pd.read_csv(cls_dir / "test_prob.csv")
                df_pred = pd.read_csv(cls_dir / "pred_pr.csv")
                df_pred = df_pred.set_index("study_id").loc[df_true["study_id"]].reset_index()
                for label in labels:
                    fpr, tpr, _ = roc_curve(df_true[label], df_prob[label])
                    auroc = auc(fpr, tpr)
                    precision, recall, _ = precision_recall_curve(df_true[label], df_prob[label])
                    auprc = auc(recall, precision)
                    f1 = f1_score(df_true[label], df_pred[label])
                    results.append({
                        "dataset": dataset,
                        "section": section,
                        "model": model,
                        "labeler": labeler,
                        "label": label,
                        "auroc": auroc,
                        "auprc": auprc,
                        "f1": f1,
                    })
results = pd.DataFrame(results)

In [ ]:
f1_results = results.set_index(["model", "labeler", "dataset", "section"])[["label", "f1"]].pivot(columns="label", values="f1")

In [ ]:
f1_results.columns.name = None
f1_results.index.names = [n.title() for n in f1_results.index.names]

In [ ]:
print(f1_results.to_latex(float_format="%.2f").replace("Enlarged Cardiomediastinum", "Enl. Card."))

# Plot Filtered Similarity Rank

In [ ]:
import sys
from pathlib import Path
from collections import defaultdict

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

sys.path.append("../rrg")
from _data import get_per_study_data, get_split_features, get_split_samples

In [ ]:
chexpert_root = Path("/opt/gpudata/chexpertplus")
mimic_root = Path("/opt/gpudata/mimic-cxr")
exp_dir = Path("/opt/gpudata/labrag")
label_dir = Path("/opt/gpudata/cxr-derived")

img_key = "img_proj"
datasets = ["mimic"]
sections = ["findings", "impression"]
fs = ["exact", "partial"]

dataset_2_model = {
    "chexpertplus": "gloria",
    "mimic": "biovilt",
}
dataset_2_csvs = {
    "chexpertplus": (
        chexpert_root / "split.csv",
        chexpert_root / "metadata.csv",
        chexpert_root / "report.csv",
    ),
    "mimic": (
        mimic_root / "mimic-cxr-2.0.0-split.csv",
        mimic_root / "mimic-cxr-2.0.0-metadata.csv",
        mimic_root / "mimic_cxr_sectioned.csv",
    ),
}

split_remap = {
    "train": "retrieval",
    "validate": "retrieval",
    "test": "inference",
}

In [ ]:
dfs = defaultdict(lambda: defaultdict(dict))
for dataset in datasets:
    print(dataset)
    h5_path = exp_dir / f"{dataset}-{dataset_2_model[dataset]}.h5"
    split_csv, metadata_csv, report_csv = dataset_2_csvs[dataset]
    for section in sections:
        print(section)
        all_df = get_per_study_data(
            split_csv=split_csv,
            metadata_csv=metadata_csv,
            label_csv=label_dir / f"{dataset}-{section}-labels-chexbert.csv",
            report_csv=report_csv,
            split_remap=split_remap,
        )
        split_samples = get_split_samples(sample_df=all_df)
        features = get_split_features(
            feature_h5=h5_path,
            feature_key=img_key,
            sample_df=all_df,
        )

        retrieval_df = split_samples["retrieval"]
        inference_df = split_samples["inference"]
        retrieval_features = features["retrieval"]
        inference_features = features["inference"]
        retrieval_df["study_id"] = retrieval_df["study_id"].astype(str)
        inference_df["study_id"] = inference_df["study_id"].astype(str)
        for f in fs:
            print(f)
            res = pd.read_csv(exp_dir / f"exp-{dataset}" / f"exp-{section}" / "exp-filter" / f"{section}_top-5_{dataset_2_model[dataset]}-chexbert-pr-pred-label_{f}_simple_Mistral-7B-Instruct-v0.3.csv")
            res["temp"] = res["retrieved_studies"].str.split(", ")
            df = res[["study_id", "temp"]].explode("temp")
            df["study_id"] = df["study_id"].astype(str)
            df = df.dropna()

            inference_idxs = inference_df.reset_index().set_index("study_id").loc[df["study_id"], "index"].to_numpy()
            retrieval_idxs = retrieval_df.reset_index().set_index("study_id").loc[df["temp"], "index"].to_numpy()

            sims = cosine_similarity(inference_features, retrieval_features)
            sort_idxs = sims.argsort(axis=1)
            ranks = sort_idxs.argsort(axis=1)

            retrieved_ranks = (len(retrieval_features)-1) - ranks[inference_idxs, retrieval_idxs]
            df["retrieved_rank"] = retrieved_ranks
            dfs[dataset][section][f] = (df, res["study_id"].nunique())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cmap = sns.color_palette(palette="Set3")
cmap

In [ ]:
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(6, 3))

bins = list(range(27))
bins[-1] = 1000000

findings_exact, findings_n = dfs["mimic"]["findings"]["exact"]
findings_partial, _ = dfs["mimic"]["findings"]["partial"]

impression_exact, impression_n = dfs["mimic"]["impression"]["exact"]
impression_partial, _ = dfs["mimic"]["impression"]["partial"]

sns.histplot(findings_exact["retrieved_rank"], bins=bins, ax=ax1, color=cmap[4], linewidth=1, zorder=10, alpha=1, edgecolor="grey")
sns.histplot(findings_partial["retrieved_rank"], bins=bins, ax=ax3, color=cmap[1], linewidth=1, zorder=10, alpha=1, edgecolor="grey")

sns.histplot(impression_exact["retrieved_rank"], bins=bins, ax=ax2, color=cmap[4], linewidth=1, zorder=10, alpha=1, edgecolor="grey")
sns.histplot(impression_partial["retrieved_rank"], bins=bins, ax=ax4, color=cmap[1], linewidth=1, zorder=10, alpha=1, edgecolor="grey")

ax1.set_title(f"Exact Filter, Findings, N={findings_n}", fontsize=10)
ax3.set_title(f"Partial Filter, Findings, N={findings_n}", fontsize=10)

ax2.set_title(f"Exact Filter, Impression, N={impression_n}", fontsize=10)
ax4.set_title(f"Partial Filter, Impression, N={impression_n}", fontsize=10)

ax1.set_ylim([0, 550])
ax2.set_ylim([0, 550])
ax3.set_ylim([0, 550])
ax4.set_ylim([0, 550])

ax1.set_xlim([0, 25])
ax2.set_xlim([0, 25])
ax3.set_xlim([0, 25])
ax4.set_xlim([0, 25])

ax1.set_xticks([0, 5, 10, 15, 20, 25])
ax2.set_xticks([0, 5, 10, 15, 20, 25])
ax3.set_xticks([0, 5, 10, 15, 20, 25])
ax4.set_xticks([0, 5, 10, 15, 20, 25])

ax1.set_xticklabels([])
ax3.set_xticklabels([])

ax2.set_xticklabels([0, 5, 10, 15, 20, 25])
ax4.set_xticklabels([0, 5, 10, 15, 20, 25])

ax1.set_xlabel("")
ax2.set_xlabel("")
ax3.set_xlabel("Image Similarity Rank")
ax4.set_xlabel("Image Similarity Rank")

ax2.set_ylabel("")
ax4.set_ylabel("")

ax1.set_yticks([0, 100, 200, 300, 400, 500])
ax2.set_yticks([0, 100, 200, 300, 400, 500])
ax3.set_yticks([0, 100, 200, 300, 400, 500])
ax4.set_yticks([0, 100, 200, 300, 400, 500])

ax1.set_yticklabels([0, 100, 200, 300, 400, 500])
ax2.set_yticklabels([])
ax3.set_yticklabels([0, 100, 200, 300, 400, 500])
ax4.set_yticklabels([])

ax1.grid(which="major", axis="y", zorder=0)
ax2.grid(which="major", axis="y", zorder=0)
ax3.grid(which="major", axis="y", zorder=0)
ax4.grid(which="major", axis="y", zorder=0)

for ax in [ax1, ax2, ax3, ax4]:
    ax.spines["bottom"].set_zorder(20)
    ax.spines["left"].set_zorder(20)

fig.suptitle("Top 5 Filtered Image Similarity", y=.95, fontsize=10)
fig.tight_layout()

fig.savefig(f"../figs/other/selected-similarity-rank.png", dpi=1000)
fig.savefig(f"../figs/other/selected-similarity-rank.pdf")

# Inspect Generations

In [ ]:
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
from f1chexbert import F1CheXbert

m = F1CheXbert(device="cpu")

def get_radgraph_tokens(radgraph_blob: str) -> list[str]:
    dat = eval(radgraph_blob) # here be dragons
    toks = []
    for _, x in dat["entities"].items():
        toks.append(x["tokens"])
    return toks

def get_chexbert_names(pos_labels: str | list[int]) -> list[str]:
    if isinstance(pos_labels, str):
        pos_labels = eval(pos_labels) # more dragons
    pos_labels = np.asarray(pos_labels)
    return np.asarray(m.target_names)[pos_labels == 1].tolist()

In [ ]:
exp_dir = Path("/opt/gpudata/labrag")

scores = pd.read_csv(exp_dir / "exp-mimic/exp-findings/exp-core/findings_top-5_biovilt-chexbert-pr-pred-label_exact_simple_Mistral-7B-Instruct-v0.3_METRICS.csv")
generations = pd.read_csv(exp_dir / "exp-mimic/exp-findings/exp-core/findings_top-5_biovilt-chexbert-pr-pred-label_exact_simple_Mistral-7B-Instruct-v0.3.csv")

cxrmate_scores = pd.read_csv(exp_dir / "exp-baselines/cxrmate_findings_METRICS.csv")
cxrmate_text = pd.read_csv(exp_dir / "exp-baselines/cxrmate_findings.csv")

### Actual

In [ ]:
temp = "AP view of the chest.  Right PICC is seen with tip at the upper SVC."
mask = generations["actual_text"].str.contains(temp)
assert mask.sum() == 1

idx = generations.index[mask][0]
study_id = generations.loc[idx, "study_id"]
print(generations.loc[idx, "actual_text"])

In [ ]:
get_chexbert_names(scores.loc[idx, "actual_chexbert"])

In [ ]:
get_chexbert_names(m.get_label("PICC"))

In [ ]:
get_chexbert_names(m.get_label("Cardiac silhouette appears moderately enlarged"))

In [ ]:
get_radgraph_tokens(scores.loc[idx, "actual_radgraph"])

### LaB-RAG

In [ ]:
scores.loc[idx, ["f1radgraph", "f1chexbert"]]

In [ ]:
print(generations.loc[idx, "generated_text"])

In [ ]:
get_chexbert_names(scores.loc[idx, "generated_chexbert"])

In [ ]:
get_chexbert_names(m.get_label("internal jugular catheter"))

In [ ]:
get_chexbert_names(m.get_label("cardiomegaly"))

In [ ]:
get_radgraph_tokens(scores.loc[idx, "generated_radgraph"])

### CXRMate

In [ ]:
cxrmate_mask = cxrmate_scores["study_id"] == study_id
assert cxrmate_mask.sum() == 1
cxrmate_idx = cxrmate_scores.index[mask][0]

In [ ]:
cxrmate_scores.loc[cxrmate_idx, ["f1radgraph", "f1chexbert"]]

In [ ]:
pprint(cxrmate_text.loc[cxrmate_idx, "generated_text"])

In [ ]:
get_chexbert_names(cxrmate_scores.loc[cxrmate_idx, "generated_chexbert"])

In [ ]:
get_chexbert_names(m.get_label("PICC"))

In [ ]:
get_chexbert_names(m.get_label("cardiac silhouette is enlarged"))

In [ ]:
get_radgraph_tokens(cxrmate_scores.loc[cxrmate_idx, "generated_radgraph"])

# Descriptive Stats

In [ ]:
import sys
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from tableone import TableOne

sys.path.append("../rrg")
from _data import get_per_study_data

In [ ]:
chexpert_root = Path("/opt/gpudata/chexpertplus")
mimic_root = Path("/opt/gpudata/mimic-cxr")
exp_dir = Path("/opt/gpudata/labrag")
label_dir = Path("/opt/gpudata/cxr-derived")

datasets = ["chexpertplus", "mimic"]
sections = ["findings", "impression"]

dataset_2_csvs = {
    "chexpertplus": (
        chexpert_root / "split.csv",
        chexpert_root / "metadata.csv",
        chexpert_root / "report.csv",
    ),
    "mimic": (
        mimic_root / "mimic-cxr-2.0.0-split.csv",
        mimic_root / "mimic-cxr-2.0.0-metadata.csv",
        mimic_root / "mimic_cxr_sectioned.csv",
    ),
}

In [ ]:
dfs = defaultdict(dict)
for dataset in datasets:
    print(dataset)
    split_csv, metadata_csv, report_csv = dataset_2_csvs[dataset]
    meta = pd.read_csv(metadata_csv)
    for section in sections:
        print(section)
        df = get_per_study_data(
            split_csv=split_csv,
            metadata_csv=metadata_csv,
            label_csv=label_dir / f"{dataset}-{section}-labels-chexbert.csv",
            report_csv=report_csv,
            extra_meta_cols=["StudyDate"] if dataset == "mimic" else ["age", "sex", "race", "ethnicity"],
        )

        dfs[dataset][section] = df

#### Table 1 Helper

In [ ]:
def make_table_one(demo: pd.DataFrame) -> pd.DataFrame:
    tab1 = TableOne(
        data=demo,
        columns=["Age", "Sex", "Race"],
        categorical=["Sex", "Race"],
        continuous=["Age"],
        groupby="split",
        nonnormal=["Age"],
        order={
            "Sex": ["Female", "Male", "Unknown"],
            "Race": ["White", "Black", "Hispanic/Latino", "Asian", "AIAN", "NHPI", "Other", "Unknown"],
        }
    ).tableone

    tab1.columns = tab1.columns.get_level_values(1)
    tab1 = tab1[["Overall", "train", "validate", "test"]]

    overall_patients = demo["subject_id"].nunique()
    split_patients = demo.groupby("split")["subject_id"].nunique()
    split_patients.loc["Overall"] = overall_patients
    tab1.loc[("Count, N", "Patients"), :] = split_patients

    overall_size = len(demo)
    split_sizes = demo.groupby("split").size()
    split_sizes.loc["Overall"] = overall_size

    overall_missing_age = demo["Age"].isna().sum()
    split_missing_age = demo.loc[demo["Age"].isna()].groupby("split").size()
    split_missing_age.loc["Overall"] = overall_missing_age

    split_missing_age_percent = (split_missing_age / split_sizes * 100).apply(lambda x: f"{x:0.1f}")
    split_missing_age = split_missing_age.astype(str) + " (" + split_missing_age_percent + ")"

    tab1.loc[("Age", "Missing, N (%)"), :] = split_missing_age

    tab1 = tab1.fillna("")

    tab1.columns = tab1.columns.str.title()

    tab1.index = pd.MultiIndex.from_tuples([
        ("Count, N", "Studies"),
        ("Age", "Median [Q1, Q3]"),
        ("Sex, N (%)", "Female"),
        ("Sex, N (%)", "Male"),
        ("Sex, N (%)", "Unknown"),
        ("Race, N (%)", "White"),
        ("Race, N (%)", "Black"),
        ("Race, N (%)", "Hispanic/Latino"),
        ("Race, N (%)", "Asian"),
        ("Race, N (%)", "AIAN"),
        ("Race, N (%)", "NHPI"),
        ("Race, N (%)", "Other"),
        ("Race, N (%)", "Unknown"),
        ("Count, N", "Patients"),
        ("Age", "Missing, N (%)"),
    ])

    tab1 = tab1.loc[["Count, N", "Age", "Sex, N (%)", "Race, N (%)"]]
    return tab1

#### View Table Helper

In [ ]:
def make_view_table(df: pd.DataFrame) -> pd.DataFrame:
    view_tab = TableOne(
        data=df.replace({"ViewPosition": {"": "Unknown"}}),
        columns=["ViewPosition"],
        categorical=["ViewPosition"],
        groupby="split",
        order={
            "ViewPosition": [
                "PA",
                "AP",
                "LATERAL",
                "LL",
                "AP AXIAL",
                "AP LLD",
                "AP RLD",
                "PA RLD",
                "PA LLD",
                "LAO",
                "RAO",
                "LPO",
                "XTABLE LATERAL",
                "SWIMMERS",
                "Unknown",
            ],
        }
    ).tableone

    view_tab.columns = [x.title() for x in view_tab.columns.get_level_values(1)]
    view_tab = view_tab[["Overall", "Train", "Validate", "Test"]]
    views = view_tab.index.get_level_values(1)[1:].to_list()
    view_tab.index = ["Total"] + views
    view_tab = view_tab.loc[views + ["Total"]]
    view_tab.index.name = "View, N (%)"
    view_tab = view_tab.reset_index()
    return view_tab

#### Label Prevalence Helper

In [ ]:
labels = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Enlarged Cardiomediastinum",
    "Fracture",
    "Lung Lesion",
    "Lung Opacity",
    "No Finding",
    "Pleural Effusion",
    "Pleural Other",
    "Pneumonia",
    "Pneumothorax",
    "Support Devices",
]

def make_label_table(df: pd.DataFrame) -> pd.DataFrame:
    label_tab = TableOne(
        data=df,
        columns=labels,
        categorical=labels,
        groupby="split",
    ).tableone

    label_tab.columns = [x.title() for x in label_tab.columns.get_level_values(1)]
    label_tab = label_tab[["Overall", "Train", "Validate", "Test"]]
    label_tab = label_tab.loc[label_tab.index.get_level_values(1).isin(["", "1"])].copy()
    label_tab.index = ["Count, N"] + [f"{l}, N (%)" for l in labels]
    return label_tab

### MIMIC
requires joining back to MIMIC-IV metadata

In [ ]:
admits = pd.read_csv("/opt/gpudata/mimic/iv/hosp/admissions.csv.gz")
race = admits.sort_values("admittime").drop_duplicates("subject_id", keep="last")[["subject_id", "race"]]
race = race.set_index("subject_id")["race"]

patients = pd.read_csv("/opt/gpudata/mimic/iv/hosp/patients.csv.gz")
assert not patients["subject_id"].duplicated().any()

patients["race"] = patients["subject_id"].apply(lambda sid: race.get(sid, np.nan))

In [ ]:
def make_mimic_demo(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["StudyYear"] = pd.to_datetime(df["StudyDate"], format="%Y%m%d").dt.year

    demo = df.merge(patients, on="subject_id", how="left")[
        [
            "subject_id",
            "study_id",
            "dicom_id",
            "split",
            "ViewPosition",
            "StudyYear",
            "gender",
            "anchor_age",
            "anchor_year",
            "race",
        ]
    ]

    demo["Age"] = (demo["anchor_age"] + (demo["StudyYear"] - demo["anchor_year"]))
    demo["Sex"] = demo["gender"].replace({"M": "Male", "F": "Female"}).fillna("Unknown")
    demo["Race"] = demo["race"].replace({
        "WHITE": "White",
        "BLACK/AFRICAN AMERICAN": "Black",
        "UNKNOWN": "Unknown",
        "WHITE - OTHER EUROPEAN": "White",
        "OTHER": "Other",
        "HISPANIC/LATINO - PUERTO RICAN": "Hispanic/Latino",
        "WHITE - RUSSIAN": "White",
        "ASIAN - CHINESE": "Asian",
        "BLACK/CAPE VERDEAN": "Black",
        "HISPANIC/LATINO - DOMINICAN": "Hispanic/Latino",
        "ASIAN": "Asian",
        "BLACK/CARIBBEAN ISLAND": "Black",
        "BLACK/AFRICAN": "Black",
        "PORTUGUESE": "White",
        "ASIAN - SOUTH EAST ASIAN": "Asian",
        "WHITE - EASTERN EUROPEAN": "White",
        "HISPANIC/LATINO - GUATEMALAN": "Hispanic/Latino",
        "ASIAN - ASIAN INDIAN": "Asian",
        "AMERICAN INDIAN/ALASKA NATIVE": "AIAN",
        "HISPANIC OR LATINO": "Hispanic/Latino",
        "WHITE - BRAZILIAN": "White",
        "HISPANIC/LATINO - SALVADORAN": "Hispanic/Latino",
        "HISPANIC/LATINO - COLUMBIAN": "Hispanic/Latino",
        "HISPANIC/LATINO - HONDURAN": "Hispanic/Latino",
        "HISPANIC/LATINO - CUBAN": "Hispanic/Latino",
        "HISPANIC/LATINO - CENTRAL AMERICAN": "Hispanic/Latino",
        "SOUTH AMERICAN": "Hispanic/Latino",
        "UNABLE TO OBTAIN": "Unknown",
        "HISPANIC/LATINO - MEXICAN": "Hispanic/Latino",
        "ASIAN - KOREAN": "Asian",
        "PATIENT DECLINED TO ANSWER": "Unknown",
        "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER": "NHPI",
        "MULTIPLE RACE/ETHNICITY": "Other",
    }).fillna("Unknown")

    return demo

In [ ]:
mimic_findings_tab1 = make_table_one(make_mimic_demo(dfs["mimic"]["findings"]))
mimic_impression_tab1 = make_table_one(make_mimic_demo(dfs["mimic"]["impression"]))

In [ ]:
mimic_findings_views = make_view_table(dfs["mimic"]["findings"])
mimic_impression_views = make_view_table(dfs["mimic"]["impression"])

In [ ]:
mimic_findings_labels = make_label_table(dfs["mimic"]["findings"])
mimic_impression_labels = make_label_table(dfs["mimic"]["impression"])

### CheXpert Plus

In [ ]:
def make_chexpert_demo(df: pd.DataFrame) -> pd.DataFrame:
    demo = df.copy()

    # combined race/ethnicity as per US OMB 2024 update
    demo["Race"] = demo["ethnicity"].where(demo["ethnicity"] == "Hispanic/Latino", other=demo["race"])

    demo["Race"] = demo["Race"].replace({
        "Pacific Islander": "NHPI",
        "Native American": "AIAN",
        "Patient Refused": "Unknown",
    })

    demo["Sex"] = demo["sex"].copy()
    demo["Age"] = demo["age"].copy()

    return demo

In [ ]:
chexpert_findings_tab1 = make_table_one(make_chexpert_demo(dfs["chexpertplus"]["findings"]))
chexpert_impression_tab1 = make_table_one(make_chexpert_demo(dfs["chexpertplus"]["impression"]))

In [ ]:
chexpert_findings_views = make_view_table(dfs["chexpertplus"]["findings"])
chexpert_impression_views = make_view_table(dfs["chexpertplus"]["impression"])

In [ ]:
chexpert_findings_labels = make_label_table(dfs["chexpertplus"]["findings"])
chexpert_impression_labels = make_label_table(dfs["chexpertplus"]["impression"])

### Save TeX Tables

In [ ]:
for dataset in ["chexpert", "mimic"]:
    for section in ["findings", "impression"]:
        for table in ["tab1", "views", "labels"]:
            with open(f"../figs/other/{table}-{dataset}-{section}.tex", "w") as f:
                df = eval(f"{dataset}_{section}_{table}") # lazy man's solution
                f.write(mimic_findings_tab1.to_latex())